In [ ]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime, timedelta
import time 

URL = "https://www.oref.org.il//Shared/Ajax/GetAlarmsHistory.aspx"
FROM_DATE = '7.10.23'
TO_DATE = '11.11.23'
DAYS_INTERVAL = 2
CSV_FILE_PATH = "C:/Projects/Python/Projects/BestTimeToShower/data/data.csv"
SLEEP_TIME = 5

In [ ]:
def get_user_settlement():
    user_settlment = input("Please input the settlement where you would like to map the distribution of alarms\n"
                           "here ---> ")
    return user_settlment

def get_request(current_date, dest_date, alerts_list):
    try:    
        params = {'lang':'en',
                    'fromDate': current_date,
                    'toDate': dest_date,
                    'mode': '0'}
        response = requests.get(URL, params=params)
        if response.status_code == 200:
            time.sleep(SLEEP_TIME)
            current_alerts_list = json.loads(response.text)
            missiles_alerts_list = [item for item in current_alerts_list if item['category_desc']=='Missiles']
            alerts_list += missiles_alerts_list
        else:
            response.raise_for_status()
        
    except requests.HTTPError as e:
        print(f"HTTP error occurred: {e}")    

def get_oref_alarms(from_date, to_date):
    all_time_alarams_list = []
    current_date = datetime.strptime(from_date, '%d.%m.%y')
    target_date = datetime.strptime(to_date, '%d.%m.%y')
    print(current_date)
    while current_date <= target_date:
        dest_date = current_date + timedelta(days=DAYS_INTERVAL)  
        get_request(current_date, dest_date, all_time_alarams_list)
        print(f"Get the alarms from date: {current_date} to date: {dest_date}")
        current_date += timedelta(days=DAYS_INTERVAL)
    
    difference = current_date - target_date
    days_diff = difference.days
    day_to_add = DAYS_INTERVAL-days_diff
    current_date -= timedelta(days=DAYS_INTERVAL) # return current_date to the correct date cause we add DAYS_INTERVAL at the end of while loop
    get_request(current_date, current_date + timedelta(days=day_to_add), all_time_alarams_list)
    
    df = pd.DataFrame(all_time_alarams_list)
    # Convert the data column to lowercase to make it easy to work with the dataframe in future
    df['data'] = df['data'].str.lower()
    df.to_csv(CSV_FILE_PATH)
    
     
get_oref_alarms(FROM_DATE, TO_DATE)

In [ ]:
# This func get dataframe and return a list of all the settlement
def return_data_list(dataframe):
    items_lst = dataframe['data'].tolist()
    items_lst_no_duplications = set(items_lst)
    return items_lst_no_duplications


# This function get dataframe and settlement_name and return a list of all the settlement in the df that settlement_name is insid them
def create_user_settl_lst(dataframe, settlement_name):
    given_lst = return_data_list(dataframe)
    settl_lst = []
    for item in given_lst:
        if settlement_name.lower() in item:
            if item not in settl_lst:
                settl_lst.append(item)
    return settl_lst

In [ ]:
# This function display the distribution of alarms in a specific settlement
def display_dist(df, settlement):
    settlement_lst= create_user_settl_lst(df, settlement)
    
    if settlement_lst != []:
        # Filtered the df by the given settlement lst
        filtered_df = df[df['data'].isin(settlement_lst)]
        
        # Access the 'time' column of the DataFrame
        filtered_df['time'] = pd.to_datetime(filtered_df['time'])

        # Plot histogram
        plt.hist(filtered_df['time'].dt.hour + filtered_df['time'].dt.minute / 60, bins=4 * 24, range=(0, 24), color='skyblue', edgecolor='black')

        plt.xlabel('Time')
        plt.ylabel('Alarms')
        plt.title(f"Distribution of alarms in {settlement}")
        # Set X-axis ticks and labels
        plt.xticks(range(24), labels=[str(i) for i in range(24)])
        plt.show()

    else:
        raise ValueError("You selected a Settlement that does not exist.")

In [ ]:
# This function receives df and settlement name then count the total amount of alerts in this settlement
def alerts_count(df, settlement):
    user_settlement_lst= create_user_settl_lst(df, settlement)
    
    if user_settlement_lst != []:
        # Filtered the df by the given settlement lst
        filtered_df = df[df['data'].isin(user_settlement_lst)]
        return filtered_df.shape[0]
    else:
        raise ValueError("You selected a Settlement that does not exist.")
    

# This function receives df and count the total amount of alerts in the requested date range
def total_alerts_countt(df):
    return df.shape[0]

In [ ]:
def best_time_to_shower(settle, start_time, end_time):
    
    pass

In [ ]:
class aggregator:
    
    def __init__(self, settlement, df):
        self.settlement = settlement
        self.df = df

    @staticmethod
    def get_request(current_date, dest_date, alerts_list):
        try:    
            params = {'lang':'en',
                        'fromDate': current_date,
                        'toDate': dest_date,
                        'mode': '0'}
            response = requests.get(URL, params=params)
            if response.status_code == 200:
                time.sleep(SLEEP_TIME)
                current_alerts_list = json.loads(response.text)
                missiles_alerts_list = [item for item in current_alerts_list if item['category_desc']=='Missiles']
                alerts_list += missiles_alerts_list
            else:
                response.raise_for_status()
            
        except requests.HTTPError as e:
            print(f"HTTP error occurred: {e}")    

    @staticmethod
    def get_oref_alarms(from_date, to_date):
        all_time_alarams_list = []
        current_date = datetime.strptime(from_date, '%d.%m.%y')
        target_date = datetime.strptime(to_date, '%d.%m.%y')
        print(current_date)
        while current_date <= target_date:
            dest_date = current_date + timedelta(days=DAYS_INTERVAL)  
            get_request(current_date, dest_date, all_time_alarams_list)
            print(f"Get the alarms from date: {current_date} to date: {dest_date}")
            current_date += timedelta(days=DAYS_INTERVAL)
        
        difference = current_date - target_date
        days_diff = difference.days
        day_to_add = DAYS_INTERVAL-days_diff
        current_date -= timedelta(days=DAYS_INTERVAL) # return current_date to the correct date cause we add DAYS_INTERVAL at the end of while loop
        get_request(current_date, current_date + timedelta(days=day_to_add), all_time_alarams_list)
        
        df = pd.DataFrame(all_time_alarams_list)
        # Convert the data column to lowercase to make it easy to work with the dataframe in future
        df['data'] = df['data'].str.lower()
        df.to_csv(CSV_FILE_PATH)
    


    # This func get dataframe and return a set of all the settlement in df
    def return_data_list(self):
        items_lst = self.df['data'].tolist()
        items_lst_no_duplications = set(items_lst)
        return items_lst_no_duplications


    # This function get dataframe and settlement_name and return a list of all the settlement in the df that settlement_name is insid them
    def create_user_settl_lst(self):
        given_lst = return_data_list(self.df)
        settl_lst = []
        for item in given_lst:
            if self.settlement.lower() in item:
                if item not in settl_lst:
                    settl_lst.append(item)
        return settl_lst
    


    # This function display the distribution of alarms in a specific settlement
    def display_dist(self):
        settlement_lst= create_user_settl_lst(self.df, self.settlement)
        
        if settlement_lst != []:
            # Filtered the df by the given settlement lst
            filtered_df = self.df[self.df['data'].isin(settlement_lst)]
            
            # Access the 'time' column of the DataFrame
            filtered_df['time'] = pd.to_datetime(filtered_df['time'])

            # Plot histogram
            plt.hist(filtered_df['time'].dt.hour + filtered_df['time'].dt.minute / 60, bins=4 * 24, range=(0, 24), color='skyblue', edgecolor='black')

            plt.xlabel('Time')
            plt.ylabel('Alarms')
            plt.title(f"Distribution of alarms in {self.settlement}")
            # Set X-axis ticks and labels
            plt.xticks(range(24), labels=[str(i) for i in range(24)])
            plt.show()

        else:
            raise ValueError("You selected a Settlement that does not exist.")
        

    # This function receives df and settlement name then count the total amount of alerts in this settlement
    def alerts_count(self):
        user_settlement_lst= create_user_settl_lst(self.df, self.settlement)
        
        if user_settlement_lst != []:
            # Filtered the df by the given settlement lst
            filtered_df = self.df[self.df['data'].isin(user_settlement_lst)]
            return filtered_df.shape[0]
        else:
            raise ValueError("You selected a Settlement that does not exist.")
        

    # This function receives df and count the total amount of alerts in the requested date range
    def total_alerts_countt(self):
        return self.df.shape[0]
    


In [ ]:
def main():

    ''' print((aggregator.get_alerts_count('sderot')))
    print((aggregator.get_alerts_count('mivtahim')))
    print((aggregator.plot_distribturion('sderot')))
    '''
    try:    
        dataframe = pd.read_csv(CSV_FILE_PATH) 
    
        user_inp = get_user_settlement()
        print(f"You selected: {user_inp}")
 
       
        display_dist(dataframe, user_inp)
    
        print(total_alerts_countt(dataframe))
        print(alerts_count(dataframe, user_inp))
        
       
    except ValueError as e:
        print(f"An error occured: {e}")


In [ ]:
if __name__ == "__main__":
    main()